# Predict IMDB movie rating
## part 03 Model Selection

This section is used to build, ensemble, and validate models' performance. It consists of 4 steps

1. Standardardization
2. implement dimention reducation with PCA
3. Model Training
4. Ensemble Generation

# Setup

In [21]:
#!pip install prince
#!pip install xgboost

# load libraries
import pandas as pd
import numpy as np
import prince
import xgboost as xgb
from math import exp
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [9]:
# read dataset
feature_path = 'https://raw.githubusercontent.com/Carloszone/IMDB_Rating_Prediction/main/features.csv'
target_path = 'https://raw.githubusercontent.com/Carloszone/IMDB_Rating_Prediction/main/dependent_var.csv'
X = pd.read_csv(feature_path)
Y = pd.read_csv(target_path).values.ravel()

In [10]:
numeric_vars = ['num_critic_for_reviews', 'duration', 'director_facebook_likes', 'actor_3_facebook_likes', \
                'actor_1_facebook_likes', 'gross', 'num_voted_users', 'cast_total_facebook_likes', \
                'facenumber_in_poster', 'num_user_for_reviews', 'budget', 'actor_2_facebook_likes', 'movie_facebook_likes']
label_vars =[]
for col in X.columns.tolist():
    if col not in numeric_vars:
        label_vars.append(col)
        
# X[label_vars] = X[label_vars].astype(str)

# Standardization

In [11]:
def strandardization(df, X, numeric_vars = numeric_vars):
    df = df.copy()
    scaler = preprocessing.StandardScaler().fit(X[numeric_vars])
    df.loc[:,numeric_vars] = scaler.transform(df[numeric_vars])
    return df

# Model Training

1. linear model + lasso
2. SVM
3. Random Forest
4. Gradient Boosting(XGB)

In [12]:
# function to implement grid search
def grid_search(models_parameters,X, y):
    if len(models_parameters['models']) != len(models_parameters['parameters']):
        return 'models and parameters have different length'
    
    params = []
    for ind in range(len(models_parameters['models'])):
        model = models_parameters['models'][ind]
        estimator = models_parameters['estimators'][ind]
        parameter = models_parameters['parameters'][ind]
        
        clf = GridSearchCV(estimator = estimator, param_grid = parameter, verbose = 20 ,scoring = 'neg_mean_squared_error')
        clf.fit(X,y)
        params.append(clf.best_params_)
        print('Model:', model)
        print('Best params', clf.best_params_)
    models_parameters['Best_params'] = params
    return models_parameters

# function to implement cross validation and get result for all models
def cross_validation(models, X, y, cv = 5):
    if len(models['model']) != len(models['model_name']):
        return 'length error'
    
    res = []
    for ind in range(len(models['model_name'])):
        print('Model:',models['model_name'][ind] )
        cv_results = cross_validate(models['model'][ind], X, y, cv=cv, scoring = 'neg_mean_squared_error')
        res.append(abs(cv_results['test_score'].mean()))
    cv_res = pd.DataFrame({'Model': models['model_name'], 'CV_result': res})
    return cv_res

In [13]:
# get trainset and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=721)

X_train_s = strandardization(X_train, X_train).to_numpy()
X_test_s = strandardization(X_test, X_train).to_numpy()

In [14]:
# first round gird search
models = ['lasso', 'SVM', 'RandomForest', 'XGB']
estimators = [Lasso(), LinearSVR(), RandomForestRegressor(), XGBRegressor(learning_rate=0.1)]
parameters = [{'alpha':[exp(-10), exp(-8), exp(-4)], 'max_iter': [300,400,500], 'random_state':[721]},
             {'C':[0.001,0.005,.01], 'max_iter':[20,30,40], 'random_state':[721], \
              'loss':['squared_epsilon_insensitive'], 'dual':[False]},
             {'n_estimators':[1300,1400,1500], 'max_depth': [40,50,60], 'min_samples_split':[1,2,3],\
              'min_samples_leaf': [2,3,4], 'max_features':[15, 17, 'sqrt'], 'random_state':[721]},
             {'n_estimators':[200,300,400], 'max_depth': [30,40,50], 'colsample_bytree':[.7,.8,.9], 'random_state':[721]}]


models_parameters ={'models': models, 'estimators': estimators, 'parameters': parameters}
best_params = grid_search(models_parameters,X_train_s, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START alpha=4.5399929762484854e-05, max_iter=300, random_state=721
[CV 1/5; 1/9] END alpha=4.5399929762484854e-05, max_iter=300, random_state=721;, score=-0.849 total time=   0.0s
[CV 2/5; 1/9] START alpha=4.5399929762484854e-05, max_iter=300, random_state=721
[CV 2/5; 1/9] END alpha=4.5399929762484854e-05, max_iter=300, random_state=721;, score=-0.841 total time=   0.0s
[CV 3/5; 1/9] START alpha=4.5399929762484854e-05, max_iter=300, random_state=721
[CV 3/5; 1/9] END alpha=4.5399929762484854e-05, max_iter=300, random_state=721;, score=-0.948 total time=   0.0s
[CV 4/5; 1/9] START alpha=4.5399929762484854e-05, max_iter=300, random_state=721
[CV 4/5; 1/9] END alpha=4.5399929762484854e-05, max_iter=300, random_state=721;, score=-0.875 total time=   0.0s
[CV 5/5; 1/9] START alpha=4.5399929762484854e-05, max_iter=300, random_state=721
[CV 5/5; 1/9] END alpha=4.5399929762484854e-05, max_iter=300, random_state=721;, sc

C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1097.8053392805102, tolerance: 0.4106871429942419
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 979.0682745080815, tolerance: 0.4055350738963532
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1068.6996740500897, tolerance: 0.39468347184900826
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Convergence

[CV 2/5; 2/9] END alpha=4.5399929762484854e-05, max_iter=400, random_state=721;, score=-0.841 total time=   0.0s
[CV 3/5; 2/9] START alpha=4.5399929762484854e-05, max_iter=400, random_state=721
[CV 3/5; 2/9] END alpha=4.5399929762484854e-05, max_iter=400, random_state=721;, score=-0.948 total time=   0.0s
[CV 4/5; 2/9] START alpha=4.5399929762484854e-05, max_iter=400, random_state=721
[CV 4/5; 2/9] END alpha=4.5399929762484854e-05, max_iter=400, random_state=721;, score=-0.876 total time=   0.0s
[CV 5/5; 2/9] START alpha=4.5399929762484854e-05, max_iter=400, random_state=721
[CV 5/5; 2/9] END alpha=4.5399929762484854e-05, max_iter=400, random_state=721;, score=-0.916 total time=   0.0s
[CV 1/5; 3/9] START alpha=4.5399929762484854e-05, max_iter=500, random_state=721
[CV 1/5; 3/9] END alpha=4.5399929762484854e-05, max_iter=500, random_state=721;, score=-0.849 total time=   0.0s
[CV 2/5; 3/9] START alpha=4.5399929762484854e-05, max_iter=500, random_state=721


C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 844.0577871372203, tolerance: 0.39468347184900826
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1008.0247076127151, tolerance: 0.39951320626798853
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 848.2665330400825, tolerance: 0.4019935574032619
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Convergence

[CV 2/5; 3/9] END alpha=4.5399929762484854e-05, max_iter=500, random_state=721;, score=-0.841 total time=   0.0s
[CV 3/5; 3/9] START alpha=4.5399929762484854e-05, max_iter=500, random_state=721
[CV 3/5; 3/9] END alpha=4.5399929762484854e-05, max_iter=500, random_state=721;, score=-0.948 total time=   0.0s
[CV 4/5; 3/9] START alpha=4.5399929762484854e-05, max_iter=500, random_state=721
[CV 4/5; 3/9] END alpha=4.5399929762484854e-05, max_iter=500, random_state=721;, score=-0.876 total time=   0.0s
[CV 5/5; 3/9] START alpha=4.5399929762484854e-05, max_iter=500, random_state=721
[CV 5/5; 3/9] END alpha=4.5399929762484854e-05, max_iter=500, random_state=721;, score=-0.916 total time=   0.0s
[CV 1/5; 4/9] START alpha=0.00033546262790251185, max_iter=300, random_state=721
[CV 1/5; 4/9] END alpha=0.00033546262790251185, max_iter=300, random_state=721;, score=-0.848 total time=   0.0s
[CV 2/5; 4/9] START alpha=0.00033546262790251185, max_iter=300, random_state=721


...

C:\Users\carlo\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [        nan         nan         nan -0.65385605 -0.6541132  -0.65397513
 -0.65385605 -0.6541132  -0.65397513         nan         nan         nan
 -0.65576928 -0.65565055 -0.65535462 -0.65576928 -0.65565055 -0.65535462
         nan         nan         nan -0.65939534 -0.6591472  -0.65900946
 -0.65939534 -0.6591472  -0.65900946         nan         nan         nan
 -0.65455063 -0.65441403 -0.65413976 -0.65455063 -0.65441403 -0.65413976
         nan         nan         nan -0.65493704 -0.65500268 -0.65465334
 -0.65493704 -0.65500268 -0.65465334         nan         nan         nan
 -0.65904939 -0.6590737  -0.65905201 -0.65904939 -0.6590737  -0.65905201
         nan         nan         nan -0.68144384 -0.68116643 -0.68112988
 -0.68144384 -0.68116643 -0.68112988         nan         nan         nan
 -0.68705078 -0.68690761 -0.68684168 -0.68705078 -0.68

Model: RandomForest
Best params {'max_depth': 40, 'max_features': 15, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 1300, 'random_state': 721}
Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5; 1/27] START colsample_bytree=0.7, max_depth=30, n_estimators=200, random_state=721
[CV 1/5; 1/27] END colsample_bytree=0.7, max_depth=30, n_estimators=200, random_state=721;, score=-0.657 total time=   2.9s
[CV 2/5; 1/27] START colsample_bytree=0.7, max_depth=30, n_estimators=200, random_state=721
[CV 2/5; 1/27] END colsample_bytree=0.7, max_depth=30, n_estimators=200, random_state=721;, score=-0.591 total time=   2.6s
[CV 3/5; 1/27] START colsample_bytree=0.7, max_depth=30, n_estimators=200, random_state=721
[CV 3/5; 1/27] END colsample_bytree=0.7, max_depth=30, n_estimators=200, random_state=721;, score=-0.708 total time=   2.6s
[CV 4/5; 1/27] START colsample_bytree=0.7, max_depth=30, n_estimators=200, random_state=721
[CV 4/5; 1/27] END colsample_bytree=0.

[CV 3/5; 8/27] END colsample_bytree=0.7, max_depth=50, n_estimators=300, random_state=721;, score=-0.707 total time=   2.7s
[CV 4/5; 8/27] START colsample_bytree=0.7, max_depth=50, n_estimators=300, random_state=721
[CV 4/5; 8/27] END colsample_bytree=0.7, max_depth=50, n_estimators=300, random_state=721;, score=-0.693 total time=   2.6s
[CV 5/5; 8/27] START colsample_bytree=0.7, max_depth=50, n_estimators=300, random_state=721
[CV 5/5; 8/27] END colsample_bytree=0.7, max_depth=50, n_estimators=300, random_state=721;, score=-0.704 total time=   2.7s
[CV 1/5; 9/27] START colsample_bytree=0.7, max_depth=50, n_estimators=400, random_state=721
[CV 1/5; 9/27] END colsample_bytree=0.7, max_depth=50, n_estimators=400, random_state=721;, score=-0.655 total time=   2.7s
[CV 2/5; 9/27] START colsample_bytree=0.7, max_depth=50, n_estimators=400, random_state=721
[CV 2/5; 9/27] END colsample_bytree=0.7, max_depth=50, n_estimators=400, random_state=721;, score=-0.589 total time=   2.8s
[CV 3/5; 9/2

[CV 1/5; 16/27] END colsample_bytree=0.8, max_depth=50, n_estimators=200, random_state=721;, score=-0.655 total time=   2.7s
[CV 2/5; 16/27] START colsample_bytree=0.8, max_depth=50, n_estimators=200, random_state=721
[CV 2/5; 16/27] END colsample_bytree=0.8, max_depth=50, n_estimators=200, random_state=721;, score=-0.623 total time=   2.8s
[CV 3/5; 16/27] START colsample_bytree=0.8, max_depth=50, n_estimators=200, random_state=721
[CV 3/5; 16/27] END colsample_bytree=0.8, max_depth=50, n_estimators=200, random_state=721;, score=-0.696 total time=   2.8s
[CV 4/5; 16/27] START colsample_bytree=0.8, max_depth=50, n_estimators=200, random_state=721
[CV 4/5; 16/27] END colsample_bytree=0.8, max_depth=50, n_estimators=200, random_state=721;, score=-0.686 total time=   2.7s
[CV 5/5; 16/27] START colsample_bytree=0.8, max_depth=50, n_estimators=200, random_state=721
[CV 5/5; 16/27] END colsample_bytree=0.8, max_depth=50, n_estimators=200, random_state=721;, score=-0.683 total time=   3.0s
[CV

[CV 4/5; 23/27] END colsample_bytree=0.9, max_depth=40, n_estimators=300, random_state=721;, score=-0.678 total time=   4.3s
[CV 5/5; 23/27] START colsample_bytree=0.9, max_depth=40, n_estimators=300, random_state=721
[CV 5/5; 23/27] END colsample_bytree=0.9, max_depth=40, n_estimators=300, random_state=721;, score=-0.680 total time=   4.2s
[CV 1/5; 24/27] START colsample_bytree=0.9, max_depth=40, n_estimators=400, random_state=721
[CV 1/5; 24/27] END colsample_bytree=0.9, max_depth=40, n_estimators=400, random_state=721;, score=-0.657 total time=   3.5s
[CV 2/5; 24/27] START colsample_bytree=0.9, max_depth=40, n_estimators=400, random_state=721
[CV 2/5; 24/27] END colsample_bytree=0.9, max_depth=40, n_estimators=400, random_state=721;, score=-0.616 total time=   3.3s
[CV 3/5; 24/27] START colsample_bytree=0.9, max_depth=40, n_estimators=400, random_state=721
[CV 3/5; 24/27] END colsample_bytree=0.9, max_depth=40, n_estimators=400, random_state=721;, score=-0.725 total time=   3.2s
[CV

In [15]:
# secound round grid search. find the best learning_rate for xgb model
models = ['XGB']
estimators = [XGBRegressor(n_estimators = 300, max_depth = 30, colsample_bytree = 0.9, random_state = 721 )]
parameters = [{'learning_rate': [0.01, 0.03,0.05]}]

models_parameters ={'models': models, 'estimators': estimators, 'parameters': parameters}
grid_search(models_parameters,X_train_s, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5; 1/3] START learning_rate=0.01..........................................
[CV 1/5; 1/3] END ..........learning_rate=0.01;, score=-0.848 total time=   5.5s
[CV 2/5; 1/3] START learning_rate=0.01..........................................
[CV 2/5; 1/3] END ..........learning_rate=0.01;, score=-0.750 total time=   4.8s
[CV 3/5; 1/3] START learning_rate=0.01..........................................
[CV 3/5; 1/3] END ..........learning_rate=0.01;, score=-0.854 total time=   4.7s
[CV 4/5; 1/3] START learning_rate=0.01..........................................
[CV 4/5; 1/3] END ..........learning_rate=0.01;, score=-0.802 total time=   4.7s
[CV 5/5; 1/3] START learning_rate=0.01..........................................
[CV 5/5; 1/3] END ..........learning_rate=0.01;, score=-0.816 total time=   4.8s
[CV 1/5; 2/3] START learning_rate=0.03..........................................
[CV 1/5; 2/3] END ..........learning_rate=0.03;, 

{'models': ['XGB'],
 'estimators': [XGBRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.9, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, gamma=None,
               gpu_id=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=30,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None, n_estimators=300, n_jobs=None,
               num_parallel_tree=None, predictor=None, random_state=721,
               reg_alpha=None, reg_lambda=None, ...)],
 'parameters': [{'learning_rate': [0.01, 0.03, 0.05]}],
 'Best_params': [{'learning_rate': 0.03}]}

In [16]:
# build models with the best parameters
lasso = Lasso(**best_params['Best_params'][0])
svm = LinearSVR(**best_params['Best_params'][1])
rfm = RandomForestRegressor(**best_params['Best_params'][2])
xgb = XGBRegressor(learning_rate=0.03, **best_params['Best_params'][3])

models = ['lasso', 'SVM', 'RandomForest', 'XGB']
best_models = {'model_name':models, 'model':[lasso,svm,rfm,xgb]}

In [17]:
# implement cross validation to calculate model performance
Tx = strandardization(X, X).to_numpy()
cross_validation(best_models, Tx, Y, cv = 5)

Model: lasso
Model: SVM


C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1614.972932884382, tolerance: 0.5227760867451382
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1386.5668617327044, tolerance: 0.517121088024565
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1303.6871424742938, tolerance: 0.4861072272262027
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWa

Model: RandomForest
Model: XGB


,Model,CV_result
0,lasso,0.911118
1,SVM,1.292143
2,RandomForest,0.722338
3,XGB,0.769415


In [24]:
# random forest model has the best peroformation among 4 models, check the importance
rfm2 = RandomForestRegressor(**best_params['Best_params'][2])
rfm.fit(Tx,Y)
pd.DataFrame({'feature name':X.columns.tolist(),'Importance': rfm.feature_importances_}).sort_values(by = ['Importance'], ascending=False).to_csv('importance.csv')

RandomForestRegressor(max_depth=40, max_features=15, min_samples_leaf=2,
                      n_estimators=1300, random_state=721)

# Model Ensemble

1. Blending
2. Stacking

In [18]:
class blending(object):
    def __init__(self, models):
        self.models = models
        
    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)
        
        S_train = np.zeros((X.shape[0], len(self.models)))
        S_test = np.zeros((T.shape[0], len(self.models)))
        
        for i, clf in enumerate(self.models):
            clf.fit(X,y)
            y_pred = clf.predict(X)[:]
            S_train[:, i] = y_pred
            y_pred = clf.predict(T)[:]
            S_test[:, i] = y_pred
        
        clf = LinearRegression()
        clf.fit(S_train,y)
        y_pred = clf.predict(S_test)
            
        return y_pred
    
    
class stacking(object):
    def __init__(self, n_folds, stacker, base_models):
        self.n_folds = n_folds
        self.stacker = stacker
        self.base_models = base_models
    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)
        kf = KFold(n_splits=self.n_folds, random_state=721, shuffle=True)
        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):
            S_test_i = np.zeros((T.shape[0], n_folds))
            for j, (train_idx, test_idx) in enumerate(kf.split(y)):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                # y_holdout = y[test_idx]
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_holdout)[:]
                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict(T)[:]
            S_test[:, i] = S_test_i.mean(1)
        self.stacker.fit(S_train, y)
        y_pred = self.stacker.predict(S_test)[:]
        return y_pred
    
    
def cross_validation_ensemble(ensemble, X, y, n_folds = 5, random_state = 721):
    X = np.array(X)
    y = np.array(y)
    kf = KFold(n_splits=n_folds, random_state=random_state, shuffle=True)
    res = []
    for j, (train_ind, test_ind) in enumerate(kf.split(y)):
        X_train = X[train_ind]
        X_test = X[test_ind]
        y_train =y[train_ind]
        y_test = y[test_ind]
        pred = ensemble.fit_predict(X_train, y_train, X_test)
        mse = mean_squared_error(y_test, pred)
        res.append(mse)
    print(res)
    return np.mean(res)

In [19]:
# generate blending ensemble model and stacking ensemlbe model
base_model = [lasso,svm,rfm, xgb]
stacker = RandomForestRegressor()
n_folds = 5

blending_model = blending(best_models['model'])
stacking_model = stacking(n_folds, stacker, base_model)

In [22]:
cross_validation_ensemble(blending_model, X, Y)

C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1708.2990267232715, tolerance: 0.5031090593654044
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1707.3918906904912, tolerance: 0.5052476998464689
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1688.06526289898, tolerance: 0.4914109498464687
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWa

[0.6327130415176474, 0.6126454949585429, 0.6457705409808743, 0.7074048584376565, 0.6237356367298017]


0.6444539145249045

In [23]:
cross_validation_ensemble(stacking_model, X, Y)

C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1365.7619344772802, tolerance: 0.4030173106206015
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1369.2854364295508, tolerance: 0.40441653262955857
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1374.2512628578734, tolerance: 0.4019311849008318
  model = cd_fast.enet_coordinate_descent(
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Convergenc

[0.6340361054247697, 0.6171352108495394, 0.6390467645854658, 0.7115083070624358, 0.6651886755373592]


0.653383012691914

# Conclusion

The **blending ensemble model** has the best performance score. We should choose the blending ensemble model as the model which will be deploy into production